In [65]:
import pandas as pd
import numpy as np
import gensim
import requests
from bs4 import BeautifulSoup
import time
import re

In [2]:
df = pd.read_csv('./datasets/df_cleaned.csv')

In [3]:
df.head()

,tweet_count,City,id,tweet_text,timestamp,hashtags,username,mentions,rewtweets,replies,link
0,6006,Sacramento,1059959242336145408,Vote for our future because they are Hopeful f...,2018-11-07 00:02:31+00:00,NaN,legallylendy,NaN,0,0,https://twitter.com/legallylendy/status/105995...
1,6005,Sacramento,1059959881237782528,I grew up in a family that celebrated the priv...,2018-11-07 00:05:03+00:00,NaN,jillmansfield_,NaN,0,0,https://twitter.com/jillmansfield_/status/1059...
2,6004,Sacramento,1059960170313342978,"Getting ready for our ""Bubbles and Bling"" even...",2018-11-07 00:06:12+00:00,NaN,DJsJewelry,NaN,0,0,https://twitter.com/DJsJewelry/status/10599601...
3,901,Butte County,1059960216891080704,VOTE do it for the future . . . . . #pmgridcha...,2018-11-07 00:06:23+00:00,#pmgridchallenge #instatravel #love #wanderlus...,heywoodphotog,NaN,0,0,https://twitter.com/heywoodphotog/status/10599...
4,6003,Sacramento,1059960413637500928,#ivotedtoday #vote #voteelkgrove #waveofchange...,2018-11-07 00:07:10+00:00,#ivotedtoday #vote #voteelkgrove #waveofchange,karif304,NaN,0,0,https://twitter.com/karif304/status/1059960413...


In [13]:
# counts of hashtages (strip the #)
keyterms = {}
for i in df['hashtags'].dropna():
    i = i.replace("#", '')
    for j in i.split():
        if j not in keyterms.keys():
            keyterms[j] = 1
        else:
            keyterms[j] += 1
keyterms

{'pmgridchallenge': 26,
 'instatravel': 15,
 'love': 67,
 'wanderlust': 21,
 'instagood': 29,
 'picoftheday': 62,
 'instadaily': 11,
 'flyinghorsedesigncom': 29,
 'naturephotography': 23,
 'instagram': 10,
 'photographer': 26,
 'ivotedtoday': 1,
 'vote': 8,
 'voteelkgrove': 1,
 'waveofchange': 1,
 'thegrinch2018': 1,
 'thegrinch': 2,
 'thegrinchmovie': 1,
 'animated': 1,
 'illuminationentertainment': 1,
 'remake': 1,
 'traffic': 306,
 'Voted': 2,
 'getoutandvote': 1,
 'NewMusic': 12,
 'otmpheezy': 5,
 'otmrecords': 7,
 'upnow': 2,
 'Youtube': 4,
 'soundcloud': 4,
 'listen': 2,
 'support': 4,
 'rap': 17,
 'andygrammer': 2,
 'federalist': 3,
 'visitsacramento': 29,
 'exploremidtown': 4,
 'commUNITY': 1,
 'forthekids': 1,
 'dressagyehorse': 1,
 'horsebackriding': 1,
 'horseclippersacramento': 1,
 'horseshoes': 1,
 'style': 1,
 'foryourlifestyle': 1,
 'giving': 1,
 'winning': 1,
 'yourstyle': 1,
 'dress': 1,
 'grateful': 19,
 'designer': 1,
 'drabtofab': 1,
 'govote': 2,
 'votenow': 1,
 'c

In [15]:
# use hashtag list to build dictionary of keyterms to use combined with that of the gensim model
# Import word vectors into "model."
model = gensim.models.KeyedVectors.load_word2vec_format('../lexvec.enwiki+newscrawl.300d.W.pos.vectors')


In [16]:
keyt = []
keywords = ['fire', 'smoke', 'wildfire', 'campfire', 'forest', 'evacuate', 'hell']

for word in keywords:
    kt = [x[0] for x in model.most_similar(word, topn = 25)]
    keyt+= kt
word_list = list(set(keyt))

In [19]:
important_words = []
for word in keyterms.keys():
    if word in word_list:
        important_words.append(word)
important_words

['wildfire',
 'evacuated',
 'rescue',
 'fire',
 'fires',
 'firefighters',
 'wildlife',
 'wildfires',
 'displaced',
 'clouds',
 'fuck',
 'evacuees']

In [29]:
df['key_score']

0       0
1       0
2       0
3       0
4       0
       ..
6917    0
6918    0
6919    0
6920    0
6921    0
Name: key_score, Length: 6922, dtype: int64

In [52]:
# check each tweet for keyterms and score them
keyscore = []
for row in df.index:
    keyscore.append(0)
    for word in str(df.iloc[row]['tweet_text']).split():
        if word.lower() in important_words:
            keyscore[row]+=1



69
321
427
559
575
593
593
597
628
664
716
745
746
748
797
805
811
830
848
882
888
908
908
913
913
918
919
940
940
941
941
943
943
951
972
987
1011
1019
1027
1045
1120
1143
1161
1168
1177
1198
1210
1210
1223
1236
1274
1277
1287
1293
1311
1330
1334
1335
1336
1342
1346
1347
1349
1352
1358
1361
1366
1366
1367
1377
1379
1382
1388
1427
1484
1518
1526
1537
1538
1556
1557
1557
1571
1575
1575
1577
1633
1639
1646
1655
1665
1666
1691
1692
1693
1702
1710
1726
1726
1727
1729
1740
1772
1784
1792
1802
1804
1806
1807
1817
1824
1825
1839
1860
1912
1913
1914
1921
1923
1936
1979
1979
1979
2015
2016
2022
2073
2076
2081
2082
2083
2084
2085
2121
2128
2137
2152
2167
2167
2176
2183
2197
2198
2201
2201
2203
2204
2205
2206
2208
2219
2263
2282
2286
2296
2297
2297
2298
2298
2301
2301
2305
2309
2310
2320
2320
2361
2381
2396
2398
2402
2406
2433
2441
2484
2497
2498
2556
2567
2586
2590
2604
2610
2618
2618
2623
2628
2672
2721
2828
2836
2861
2864
2896
2902
2936
2977
2981
2990
2993
3008
3008
3018
3027
3028
3031
3054
30

In [53]:
df['key_score'] = keyscore

In [58]:
df.describe()

,tweet_count,id,rewtweets,replies,key_score
count,6922.000000,6.922000e+03,6922.000000,6922.000000,6922.000000
mean,2662.182028,1.063325e+18,0.142589,0.047096,0.060243
std,1838.588052,2.004182e+15,0.772885,0.313139,0.263318
min,0.000000,1.059959e+18,0.000000,0.000000,0.000000
25%,843.000000,1.061490e+18,0.000000,0.000000,0.000000
50%,2545.500000,1.063232e+18,0.000000,0.000000,0.000000
75%,4275.750000,1.065091e+18,0.000000,0.000000,0.000000
max,6006.000000,1.066844e+18,20.000000,10.000000,4.000000


In [59]:
df[df['key_score']>0]

,tweet_count,City,id,tweet_text,timestamp,hashtags,username,mentions,rewtweets,replies,link,key_score
69,5948,Sacramento,1059990275160973317,Listening to robynkonichiwa new album #honeyal...,2018-11-07 02:05:49+00:00,#honeyalbum,Royal84Cortez,NaN,0,0,https://twitter.com/Royal84Cortez/status/10599...,1
321,5723,Sacramento,1060282760261259264,What the fuck tho? ------- #lilwayne #uproarch...,2018-11-07 21:28:03+00:00,#lilwayne #uproarchallenge #swizzbeatz #uproar...,JacquardIWyse,NaN,0,0,https://twitter.com/JacquardIWyse/status/10602...,1
427,5623,Sacramento,1060360891357638666,"I'm at Fire Wings in Fair Oaks, CA",2018-11-08 02:38:31+00:00,NaN,hondaguy_86,NaN,0,0,https://twitter.com/hondaguy_86/status/1060360...,1
559,759,"Chico, CA",1060563632864542720,"Fire in the feather river canyon. @Chico, Cali...",2018-11-08 16:04:09+00:00,NaN,deckinspectah,NaN,0,0,https://twitter.com/deckinspectah/status/10605...,1
575,757,"Chico, CA",1060579478177492992,Closed due to camp fire in #Oroville on Hwy 70...,2018-11-08 17:07:06+00:00,#Oroville #traffic,TotalTrafficSMF,NaN,0,0,https://twitter.com/TotalTrafficSMF/status/106...,1
...,...,...,...,...,...,...,...,...,...,...,...,...
6653,235,Sacramento,1066549721282592768,Roseville Hosts Town Hall Meeting for Camp Fir...,2018-11-25 04:30:43+00:00,NaN,FOX40,NaN,0,0,https://twitter.com/FOX40/status/1066549721282...,1
6677,31,"Paradise, CA",1066569764896731136,Rapid DNA Testing Used to Identify Camp Fire V...,2018-11-25 05:50:22+00:00,NaN,FOX40,NaN,1,0,https://twitter.com/FOX40/status/1066569764896...,1
6727,171,Sacramento,1066635578043699201,Little Italy Fire Dept New York FDNY T Shirts ...,2018-11-25 10:11:53+00:00,#elkgrove #sacramento #newyork #fdny #littleit...,ElkGroveDealz,NaN,0,0,https://twitter.com/ElkGroveDealz/status/10666...,1
6763,30,Butte County,1066730858403569664,Camp fire closure in #Oroville on Hwy 70 Both ...,2018-11-25 16:30:30+00:00,#Oroville #traffic,TotalTrafficSMF,NaN,0,0,https://twitter.com/TotalTrafficSMF/status/106...,1


In [75]:
locs = []
delayer = 0
for row in df.index:
    url = df.iloc[row]['link']
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'lxml')
    spans = soup.find_all('span', {'class' : 'permalink-tweet-geo-text'})
    locs.append(spans[0].text.replace("from",'').strip())
    delayer += 1
    if delayer % 5 == 0:
        time.sleep(3)
df['from_locations'] = locs

IndexError: list index out of range

In [74]:
df['text_nourl'] = [re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', i) for i in df['tweet_text']]

In [70]:
df[df['tweet_text'].isna()]

,tweet_count,City,id,tweet_text,timestamp,hashtags,username,mentions,rewtweets,replies,link,key_score
2882,3523,Sacramento,1062646273096839169,NaN,2018-11-14 09:59:49+00:00,NaN,RobbPie53606015,NaN,0,0,https://twitter.com/RobbPie53606015/status/106...,0


In [73]:
df.drop([2882], inplace=True)

In [ ]:
df.to_csv('./datasets/cleanedmore.csv', index=False)